In [1]:
import pandas as pd
from IPython.core.display import HTML
from IPython.core.display import Image 
%load_ext music21.ipython21.ipExtension
import numpy as np
import music21 as mu
import pymongo
from fractions import Fraction
%pylab inline
from pymongo import MongoClient
from bs4 import BeautifulSoup
import re
import pydash as py_
pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (15, 5)
import math
import music21 as mu
from functools import partial
import json
# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)


Populating the interactive namespace from numpy and matplotlib


In [2]:
muAllTheThingsYouAre = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAllTheThingsYouAre.xml')
muAutumnLeaves = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAutumnLeaves.xml')
muDaysOfWineAndRoses = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettDaysOfWineAndRoses.xml')
muGroovingHigh = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettGroovinHigh.xml')
muIfIWereABell = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettIfIWereABell.xml')
muSomedayMyPrinceWillCome = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettSomedayMyPrinceWillCome.xml')
muStellaByStarlight = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettStellaByStarlight.xml')
muArmandosRhumba = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/ChickCoreaArmandosRhumba.xml')

In [3]:
def getDurationsForMongoQueries(data_row):
    valueToCheck = data_row['duration']
    try:
        valueToCheck = int(valueToCheck)
    except:
        valeToCheck = -1
    if valueToCheck == 21:
        return Fraction(21,1024) #Thirty Second note triplet
    
    elif valueToCheck == 32:
        return Fraction(1,32)  #Thirty second note
    
    elif valueToCheck == 42:
        return Fraction(21,512)  #Sixteenth note triplet
    
    elif valueToCheck == 43:
        return Fraction(21,512)  #Sixteenth note triplet

    elif valueToCheck == 51:
        return Fraction(51,1024)  #sixteenth note quintuplet
    
    elif valueToCheck == 52:
        return Fraction(51,1024)  #sixteenth note quintuplet    
    
    elif valueToCheck == 56:
        return Fraction(1,18)  #eighth note triplet of quarter note triplet
    
    elif valueToCheck == 57:
        return Fraction(1,18)  #eighth note triplet of quarter note triplet
    
    elif valueToCheck == 64:
        return Fraction(1,16)       #sixteenth note
    
    elif valueToCheck == 73:
        return Fraction(73,1024)       #Eighth note septuplet
    
    elif valueToCheck == 74:
        return Fraction(73,1024)       #Eighth note septuplet
    
    elif valueToCheck == 85:
        return Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 86:
        return Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 96:   #dotted sixteenth note
        return Fraction(3,32)
    
    elif valueToCheck == 102:     #eighth note quintuplet
        return Fraction(51,512)
    
    elif valueToCheck == 103:     #eighth note quintuplet
        return Fraction(51,512)
    
    elif valueToCheck == 128:   # eighth note
        return Fraction(1,8)
    
    elif valueToCheck == 170:   #quarter note triplet
        return Fraction(1,6)
    
    elif valueToCheck == 171:   #quarter note triplet
        return Fraction(1,6)
    
    elif valueToCheck == 192:   #dotted eighth note
        return Fraction(3,16)    
    
    elif valueToCheck == 256:   #quarter note
        return Fraction(1,4)

    elif valueToCheck == 342:   #half note triplet
        return Fraction(1,3)
    
    elif valueToCheck == 384:   #dotted quarter note
        return Fraction(3,8)
    
    elif valueToCheck == 512:   #half note
        return Fraction(1,2)
    
    elif valueToCheck == 768:   #dotted half note
        return Fraction(3,4)
    
    elif valueToCheck == 1024:   #whole note
        return Fraction(1,1)
    
    else:
        print 'hello'
        return Fraction(1)


    
def getDurationNamesForMongoQueries(data_row):
    valueToCheck = data_row['duration']
    try:
        valueToCheck = int(valueToCheck)
    except:
        valeToCheck = -1
    if valueToCheck == 21:
        return 'Thirtysecond note triplet' # Fraction(21,1024) #Thirty Second note triplet
    
    elif valueToCheck == 32:
        return 'Thirtysecond note' #Fraction(1,32)  #Thirty second note
    
    elif valueToCheck == 42:
        return 'Sixteenth note triplet' #Fraction(21,512)  #Sixteenth note triplet

    elif valueToCheck == 43:
        return 'Sixteenth note triplet' #Fraction(21,512)  #Sixteenth note triplet
    
    elif valueToCheck == 51:
        return 'Sixteenth note quintuplet' #Fraction(51,1024)  #sixteenth note quintuplet
    
    elif valueToCheck == 52:
        return 'Sixteenth note quintuplet' #Fraction(51,1024)  #sixteenth note quintuplet
    
    
    elif valueToCheck == 56:
        return 'Eighth note triplet of quarter note triplet' #Fraction(51,1024)
    elif valueToCheck == 57:
        return 'Eighth note triplet of quarter note triplet' #Fraction(51,1024)
    
    elif valueToCheck == 64:
        return 'Sixteenth note' #Fraction(1,16)       #sixteenth note
    
    elif valueToCheck == 73:
        return 'Eighth note septuplet' #Fraction(73,1024)       #sixteenth note
    
    elif valueToCheck == 74:
        return 'Eighth note septuplet' #Fraction(73,1024)       #sixteenth note
    
    elif valueToCheck == 85:
        return 'Eighth note triplet' # Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 86:
        return 'Eighth note triplet' # Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 96:   #dotted sixteenth note
        return 'Dotted sixteenth note' #Fraction(3,32)
    
    elif valueToCheck == 102:     #eighth note quintuplet
        return 'Eighth note quintuplet' #Fraction(51,512)

    elif valueToCheck == 103:     #eighth note quintuplet
        return 'Eighth note quintuplet' #Fraction(51,512)
    
    elif valueToCheck == 128:   # eighth note
        return 'Eighth note' #Fraction(1,8)
    
    elif valueToCheck == 170:   #quarter note triplet
        return 'Quarter note triplet' #Fraction(1,6)
    
    elif valueToCheck == 171:   #quarter note triplet
        return 'Quarter note triplet' #Fraction(1,6)
    
    elif valueToCheck == 192:   #dotted eighth note
        return 'Dotted eighth note' #Fraction(3,16)    
    
    elif valueToCheck == 256:   #quarter note
        return 'Quarter note' #Fraction(1,4)
    
    elif valueToCheck == 342:   #dotted quarter note
        return 'Half note triplet'

    elif valueToCheck == 384:   #dotted quarter note
        return 'Dotted quarter note' #Fraction(3,8)
    
    elif valueToCheck == 512:   #half note
        return 'Half note' #Fraction(1,2)
    
    elif valueToCheck == 768:   #dotted half note
        return 'Dotted half note' #Fraction(3,4)
    
    elif valueToCheck == 1024:   #whole note
        return 'Whole note' #Fraction(1,1)
    
    else:
        return Fraction(-1,1)


def cleanAccidental(data_row):
    accidentalString = str(data_row['accidental'])
    test = re.sub('<[^>]*>', '', accidentalString)
    return str(test) 

# def cleanNoteName(data_row):
#     accidentalString = str(data_row['noteName'])
#     test = re.sub('<[^>]*>', '', accidentalString)
#     return str(test) 

def cleanOctave(data_row):
    accidentalString = str(data_row['Octave'])
    test = re.sub('<[^>]*>', '', accidentalString)
    return str(test) 

def cleanRest(data_row):
    restVal = -1
    restAsString = str(data_row['Rest or note flag'])
    if restAsString != 'None':
        restVal ='rest'
    else:
        restVal = 'note'

    return str(restVal)

def cleanPart(data_row):
    accidentalString = str(data_row['instrument'])
    test = accidentalString[3:5]
    return str(test) 

def convertDurationAsPerQuarterNoteIsOneBeat(data_row):
    asFloat = float(data_row['duration'])
    #print asFloat
    asFloat = asFloat / 256
    asFloat = floor(asFloat * 100) / 100.0
    
    return round(asFloat, 2)



def midiNumberAssign(data_row):
    offsetForC = 0
    offsetForD = 2
    offsetForE = 4
    offsetForF = 5
    offsetForG = 7
    offsetForA = 9
    offsetForB = 11

    def adjustForAccidental(accidentalVal):
        if accidentalVal == 'None':
            return 0
        else:
            return int(accidentalVal)

    if data_row['Octave'] != "None":
        baseVal = data_row['Octave'] * 12
        if data_row['Note name'] == 'C':
            return baseVal + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'D':
            return baseVal + offsetForD + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'E':
            return baseVal + offsetForE + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'F':
            return baseVal + offsetForF + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'G':
            return baseVal + offsetForG + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'A':
            return baseVal + offsetForA + adjustForAccidental(data_row['accidental'])
        elif data_row['Note name'] == 'B':
            return baseVal + offsetForB + adjustForAccidental(data_row['accidental'])



def adjustForAccidental(accidentalVal):
    if accidentalVal == 'None':
        return 0
    else:
        return int(accidentalVal)

    if data_row['Octave'] != "None":
        baseVal = int(data_row['Octave']) * 12
        if data_row['noteName'] == 'C':
            return baseVal + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'D':
            return baseVal + offsetForD + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'E':
            return baseVal + offsetForE + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'F':
            return baseVal + offsetForF + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'G':
            return baseVal + offsetForG + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'A':
            return baseVal + offsetForA + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'B':
            return baseVal + offsetForB + adjustForAccidental(data_row['accidental'])

def fixTiedNotes(data_row):
    tieFlagString = str(data_row['tieFlag'])
    durationToAdd = -1
    if tieFlagString == '<tie type="stop"></tie>':
        durationToAdd = data_row['Duration as a fraction of a whole note']
    else:
        durationToAdd = 0

    return durationToAdd


def getChordRootAsInt(data_row):
    x = data_row['Chord name']
    if x != -1:    
        chordLetterVal = data_row['Chord name'][0:2]
        #print chordLetterVal
        if chordLetterVal.strip() == 'C':
            return 0
        elif chordLetterVal.strip() =='C#':
            return 1
        elif chordLetterVal.strip() == 'Db':
            return 1
        elif chordLetterVal.strip() == 'D':
            return 2
        elif chordLetterVal.strip() == 'D#':
            return 3
        elif chordLetterVal.strip() == 'Eb':
            return 3
        elif chordLetterVal.strip() == 'E':
            return 4
        elif chordLetterVal.strip() == 'F':
            return 5
        elif chordLetterVal.strip() == 'F#':
            return 6
        elif chordLetterVal.strip() == 'Gb':
            return 6
        elif chordLetterVal.strip() == 'G':
            return 7
        elif chordLetterVal.strip() == 'G#':
            return 8
        elif chordLetterVal.strip() == 'Ab':
            return 8
        elif chordLetterVal.strip() == 'A':
            return 9
        elif chordLetterVal.strip() == 'A#':
            return 10
        elif chordLetterVal.strip() == 'Bb':
            return 10
        elif chordLetterVal.strip() == 'B':
            return 11
        else:
            return -1
       


    
def getChordTypeAsInt(data_row):
    x = data_row['Chord name']
    #print x
    if x != -1:
        chordType = re.findall('\s(.*)', x)
        try:
            chordType = str(chordType[0])
        except:
            chordType = 'fix'
        
        if chordType == 'maj7':
            return 0
        elif chordType == 'min7':
            return 1
        elif chordType == 'dom7':
            return 2
        elif chordType == 'dim7':
            return 3
        elif chordType == 'maj7b5':
            return 4
        elif chordType == 'maj7#5':
            return 5
        elif chordType == 'maj7sus4':
            return 6
        elif chordType == 'min7b5':
            return 7
        elif chordType == 'dom7b5':
            return 8
        elif chordType == 'dom7#5':
            return 9
        elif chordType == 'dom7sus4':
            return 10
        
        return -1


def chordTypeForDisplay(data_row):
    x = data_row['Chord name']
    if x != -1:
        chordType = re.findall('\s(.*)', x)   
        try:
            chordType = str(chordType[0])
        except:
            chordType = 'fix'
        return chordType
    
    
def chordBassAsInt(data_row):
    x = data_row['Chord name']
    if x != -1:
        x = str(x)
        indexOfBackslash = x.find('/')
        if indexOfBackslash == -1:
            return -1
        else:
            #function to deal with bass note
            return 'placeholder'
    

def concatenateNoteNamesAndAccidentals(data_row):
    retString = str(data_row['Note name'])
    octave = ''
    if data_row['accidental'] == '-1':
        retString = retString + 'b'
    elif data_row['accidental'] == '1':
        retString = retString + '#'
    if not np.isnan(data_row['Octave']):
        octave = int(data_row['Octave'])
        
    retString = retString + str(octave)
    return retString


def concatenateNoteNamesAndAccidentalsNoOctave(data_row):
    retString = str(data_row['Note name'])
    if data_row['accidental'] == '-1':
        retString = retString + 'b'
    elif data_row['accidental'] == '1':
        retString = retString + '#'

    return retString

def getFloorValueForMeasure(data_row):
    offsetForMeasure = 1
    floorValue = data_row['Location of note or rest']
    floorValue = (floorValue / 4) + offsetForMeasure
    return math.floor(floorValue)

def fixAccidentalEquivalencies(data_row):
    currentNoteName = data_row['Name of note without octave']
    if not np.isnan(data_row['Octave']):
        octave = int(data_row['Octave'])
    if currentNoteName == 'None':
        return 'None'
    if currentNoteName == 'C#':
        currentNoteName = 'Db'
    if currentNoteName == 'D#':
        currentNoteName = 'Eb'
    if currentNoteName == 'F#':
        currentNoteName = 'Gb'
    if currentNoteName == 'G#':
        currentNoteName = 'Ab'
    if currentNoteName == 'A#':
        currentNoteName = 'Bb'
    if currentNoteName == 'Cb':
        currentNoteName = 'B'
    if currentNoteName == 'Fb':
        currentNoteName = 'E'
    return currentNoteName + str(octave)

def fixAccidentalEquivalenciesNoOctave(data_row):
    currentNoteName = data_row['Name of note without octave']
    #currentOctave = data_row['octave']
    if currentNoteName == 'None':
        return 'None'
    if currentNoteName == 'C#':
        currentNoteName = 'Db'
    if currentNoteName == 'D#':
        currentNoteName = 'Eb'
    if currentNoteName == 'F#':
        currentNoteName = 'Gb'
    if currentNoteName == 'G#':
        currentNoteName = 'Ab'
    if currentNoteName == 'A#':
        currentNoteName = 'Bb'
    if currentNoteName == 'Cb':
        currentNoteName = 'B'
    if currentNoteName == 'Fb':
        currentNoteName = 'E'
    return currentNoteName


def calculateDistanceOfNoteToChordRoot(data_row):
    offsetForPositiveValue = 12
    midiNumber = data_row['Midi number']
    chordRoot = data_row['Chord root as minimized midi number']
    midiNumber = (midiNumber % 12) + 12
    distanceToRoot = midiNumber - chordRoot
    distanceToRoot = distanceToRoot % 12
    return distanceToRoot
    

def calculateDistanceOfNoteToChordBass(data_row):
    pass

def createReadableChordDegreeNames(data_row):
    noteRootDistance = data_row['Semitone distance between note and chord root']
    if noteRootDistance == 0:
        return 'root'
    if noteRootDistance == 1:
        return 'flat ninth'
    if noteRootDistance == 2:
        return 'ninth'
    if noteRootDistance == 3:
        return 'sharp ninth or minor third'
    if noteRootDistance == 4:
        return 'major third'
    if noteRootDistance == 5:
        return 'eleventh'
    if noteRootDistance == 6:
        return 'sharp eleventh or flat fifth'
    if noteRootDistance == 7:
        return 'fifth'
    if noteRootDistance == 8:
        return 'flat thirteenth or sharp fifth'
    if noteRootDistance == 9:
        return 'thirteenth'
    if noteRootDistance == 10:
        return 'seventh'
    if noteRootDistance == 11:
        return 'major seventh'
    return noteRootDistance

def getNextTenChords(chordRootAsIntValues, data_row):
    currentIndex = data_row.name
    currentChordRootAsInt = data_row['Chord root as minimized midi number']
    listOfAllChords = chordRootAsIntValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]

    return sliceOfChords  


def createNextTenChordRootsAsNotes(data_row):
    stringToReturn = ''
    midiNumberSequence = data_row['Next ten chord root notes as midi numbers']
    for items in midiNumberSequence:
        if items == 0.0:
            stringToReturn = stringToReturn + 'Cx'
        elif items == 1.0:
            stringToReturn = stringToReturn + 'Db'
        elif items == 2.0:
            stringToReturn = stringToReturn + 'Dx'
        elif items == 3.0:
            stringToReturn = stringToReturn + 'Eb'
        elif items == 4.0:
            stringToReturn = stringToReturn + 'Ex'
        elif items == 5.0:
            stringToReturn = stringToReturn + 'Fx'
        elif items == 6.0:
            stringToReturn = stringToReturn + 'Gb'
        elif items == 7.0:
            stringToReturn = stringToReturn + 'Gx'
        elif items == 8.0:
            stringToReturn = stringToReturn + 'Ab'
        elif items == 9.0:
            stringToReturn = stringToReturn + 'Ax'
        elif items == 10.0:
            stringToReturn = stringToReturn + 'Bb'
        elif items == 11.0:
            stringToReturn = stringToReturn + 'Bx'
    return stringToReturn

def createNextTenChordRootsAsDegrees(data_row):
    stringToReturn = ''
    midiNumberSequence = data_row['Next ten chord root notes as midi numbers']
    firstValue = midiNumberSequence[0]
    for eachItem in midiNumberSequence:
        difference = int(eachItem) - int(firstValue)
        if difference < 0:
            difference = difference + 12
        if difference < 10:
            difference = '0' + str(difference)
        else: 
            difference = str(difference)
        stringToReturn = stringToReturn + str(difference)
    return stringToReturn


def getNextTenChordTypes(chordTypeValues, data_row):
    currentIndex = data_row.name
    currentChordType = data_row['Chord type name']
    listOfAllChords = chordTypeValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]
    sliceOfChordsAsString = ''
    for eachValue in sliceOfChords:
        sliceOfChordsAsString = sliceOfChordsAsString + eachValue
    
    #return sliceOfChords  
    return sliceOfChordsAsString

def getNextTenChordTypesV2(chordTypeValues, data_row):
    currentIndex = data_row.name
    currentChordType = data_row['Chord type name']

    listOfAllChords = chordTypeValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]
    sliceOfChordsAsString = ''
    for eachValue in sliceOfChords:
        paddedValue= eachValue.ljust(8, 'x')
        sliceOfChordsAsString = sliceOfChordsAsString + paddedValue
    
    #return sliceOfChords  
    return sliceOfChordsAsString


def getNextFiftyMidiNumbers(midiNumbersAsValues, data_row):
    currentIndex = data_row.name
    listOfAllMidiNumbers = midiNumbersAsValues
    sliceOfMidiNumbers = listOfAllMidiNumbers[currentIndex:]
    sliceOfMidiNumbers = sliceOfMidiNumbers.tolist()
    sliceOfMidiNumbers = sliceOfMidiNumbers[0:50]
    #print sliceOfMidiNumbers
    sliceOfMidiNumbersAsString = ''
    for eachValue in sliceOfMidiNumbers:
        if not (math.isnan(eachValue)):
            sliceOfMidiNumbersAsString = sliceOfMidiNumbersAsString + str(int(eachValue))
        else: 
            sliceOfMidiNumbersAsString = sliceOfMidiNumbersAsString + 'xx'
    

    return sliceOfMidiNumbersAsString




def getNextFiftyDurations(durationsAsValues, data_row):
    currentIndex = data_row.name
    listOfAllDurations = durationsAsValues
    sliceOfDurations = listOfAllDurations[currentIndex:]
    sliceOfDurations = sliceOfDurations.tolist()
    sliceOfDurations = sliceOfDurations[0:10]
    
    sliceOfDurationsAsString = ''
    for eachValue in sliceOfDurations:
        if not (math.isnan(eachValue)):
            paddedValue = str(eachValue)
            paddedValue = paddedValue.ljust(9, 'x')
            sliceOfDurationsAsString = sliceOfDurationsAsString + paddedValue
    
    return sliceOfDurationsAsString



def getNextFiftyLocations(locationsAsValues, data_row):
    currentIndex = data_row.name
    amountToNegate = data_row['Location of note or rest']
    listOfAllLocations = locationsAsValues
    sliceOfLocations = listOfAllLocations[currentIndex:]
    sliceOfLocations = sliceOfLocations.tolist()
    sliceOfLocations = sliceOfLocations[0:50]
    sliceOfLocationsAsString = ''
    for eachValue in sliceOfLocations:
        if not (math.isnan(eachValue)):
            offsetValue = eachValue - amountToNegate
            #roundedLocation = round(offsetValue, 4)
            paddedValue = str(offsetValue)
            paddedValue = paddedValue.ljust(9, 'x')
            sliceOfLocationsAsString = sliceOfLocationsAsString + paddedValue

    return sliceOfLocationsAsString.replace('False','0xxxx')


def getNextFiftyNotesAsDistancesFromStartingNote(midiNumbersAsValues, data_row):
    currentIndex = data_row.name
    listOfAllMidiNumbers = midiNumbersAsValues
    sliceOfMidiNumbers = listOfAllMidiNumbers[currentIndex:]
    sliceOfMidiNumbers = sliceOfMidiNumbers.tolist()
    sliceOfMidiNumbersWithoutNan = []
    
    for eachValue in sliceOfMidiNumbers:
        if not (math.isnan(eachValue)):
            sliceOfMidiNumbersWithoutNan.append(eachValue)

    stringToReturn = ''
    
    sliceOfMidiNumbersWithoutNan = sliceOfMidiNumbersWithoutNan[0:50]
    
    for i in range(0, len(sliceOfMidiNumbersWithoutNan)):
        currentValue = sliceOfMidiNumbersWithoutNan[i]
        try:
            nextValue = sliceOfMidiNumbersWithoutNan[i+1]
            difference = int(nextValue) - int(currentValue)
            differenceAsString = str(difference)
            if len(differenceAsString) == 1:
                differenceAsString = '+' + differenceAsString
            if len(differenceAsString) == 2:
                differenceAsString = differenceAsString + 'x'
            stringToReturn = stringToReturn + str(differenceAsString)
        except:
            pass

    return stringToReturn




def getCumulativeDuration(fullListOfDurationsAsFractions, data_row):
    currentIndex = data_row.name
    sumSoFar = fullListOfDurationsAsFractions[0:currentIndex].sum()
    #print 'At index', currentIndex, ' the sum is ' , sumSoFar

    return sumSoFar


def convertGlobalLocationToFloat( globalLocation, data_row):
    globalLocation = globalLocation
    locationInCurrentSong = data_row['Location of note or restV2']
    locationInCurrentSong = float(locationInCurrentSong)
    
    location = globalLocation + locationInCurrentSong
    return round(location, 8)

In [105]:
class CreateDataSetFromXMLV2:
    def __init__(self, xmlFile="", performerName="", composerName="", 
                 genre="", numberOfParts=-1, title="", yearOfComposition="", 
                 yearOfPerformance="", ongoingTempo=-1, 
                 numberOfQuarterBeatsInBar = -1, tempoChangeIsOccuring = False, 
                 globalLocation = -1):

        self.xmlFile = xmlFile
        self.genre = genre
        self.numberOfParts = numberOfParts
        self.parsedXMLData = ""
        self.df = ""
        self.performerName = performerName
        self.composerName = composerName
        self.genre = genre
        self.numberOfParts = numberOfParts
        self.title = title
        self.yearOfComposition = yearOfComposition
        self.yearOfPerformance = yearOfPerformance
        self.ongoingTempo = ongoingTempo
        self.numberOfQuarterBeatsInBar = numberOfQuarterBeatsInBar
        self.tempoChangeIsOccuring = tempoChangeIsOccuring
        self.globalLocation = globalLocation
    

    def convertXMLToDataFrame(self):
        parsedXMLData = BeautifulSoup(open(self.xmlFile))
        allNotes = parsedXMLData.findAll('measure')
        listOfNoteEvents = []
        for measure in allNotes:
            #print measure["number"]

            notes = measure.findAll('note')
            words = measure.findAll('words')
            
            for note in notes:
                measureDictionary = {}
                if note.pitch:
                    measureDictionary["Chord name"] = None
                    measureDictionary["instrument"] = measure.instrument["id"]
                    measureDictionary["Measure"] = measure["number"]
                    measureDictionary["Note name"] = note.step.text
                    measureDictionary["Octave"] = int(note.octave.text)
                    measureDictionary['duration'] = float(note.duration.text)
                    measureDictionary['accidental'] = note.alter
                    measureDictionary["Rest or note flag"] = 'note'
                    measureDictionary['tieFlag'] = str(note.tie)
                else:
                    measureDictionary["Chord name"] = None
                    measureDictionary["instrument"] = measure.instrument["id"]
                    measureDictionary["Measure"] = measure["number"]
                    measureDictionary["Note name"] = None
                    measureDictionary["Octave"] = None
                    measureDictionary['duration'] = float(note.duration.text)
                    measureDictionary['accidental'] = None
                    measureDictionary["Rest or note flag"] = 'rest'
                    measureDictionary['tieFlag'] = None
                if measureDictionary['instrument'] == 'P1-I1':
                    measureDictionary['Instrument name'] = "Piano Solo Jazz (Treble)"
                    listOfNoteEvents.append(measureDictionary)
            #print measure["number"]
            for word in range(0, len(words)):
                chordDictionary = {}
                chordDictionary['Chord name'] = words[word].text
                chordDictionary["instrument"] = measure.instrument["id"]
                chordDictionary["Instrument name"] = "Jazz Harmony Bass Note" 
                chordDictionary["Measure"] = measure["number"]
                chordDictionary["Note name"] = words[word].parent.parent.parent.note.step.text
                chordDictionary["Octave"] = int(words[word].parent.parent.parent.note.octave.text)
                chordDictionary['duration'] = float(words[word].parent.parent.parent.note.duration.text)
                chordDictionary['accidental'] = words[word].parent.parent.parent.note.alter
                chordDictionary["Rest or note flag"] = 'note'
                chordDictionary['tieFlag'] = str(words[word].parent.parent.parent.note.tie)
                
                listOfNoteEvents.append(chordDictionary)


        self.df = pd.DataFrame(listOfNoteEvents)
    

        

    def fixLocationAndDuration(self):
        self.df['Duration as a fraction of a whole note'] = self.df.apply(getDurationsForMongoQueries, axis=1)
        
        soloPart = self.df[self.df['Instrument name'] == 'Piano Solo Jazz (Treble)']
        soloPart = soloPart.reset_index(drop=True)
        harmonyPart = self.df[self.df['Instrument name'] == 'Jazz Harmony Bass Note']
        harmonyPart = harmonyPart.reset_index(drop=True)
                             
        soloPart['Location of note or rest'] = soloPart['Duration as a fraction of a whole note'].cumsum()
        harmonyPart['Location of note or rest'] = harmonyPart['Duration as a fraction of a whole note'].cumsum()
        soloPart['Location of note or rest'] = soloPart['Location of note or rest'] - soloPart.iloc[0]['Duration as a fraction of a whole note']
        harmonyPart['Location of note or rest'] = harmonyPart['Location of note or rest'] - harmonyPart.iloc[0]['Duration as a fraction of a whole note']
        
        soloPart['valueToAddToTiedNote']=soloPart.apply(fixTiedNotes, axis=1)
        soloPart['moveTieForFix'] = soloPart['valueToAddToTiedNote'].shift(-1)
        soloPart = soloPart[soloPart.tieFlag != '<tie type="stop"></tie>']        
        soloPart['Duration as a fraction of a whole note'] = soloPart['Duration as a fraction of a whole note'] + soloPart['moveTieForFix']
        
        self.df = pd.concat([soloPart, harmonyPart], axis=0)
        self.df = self.df.sort(['Location of note or rest', 'Instrument name'], ascending=[1,1])
        self.df = self.df.reset_index(drop=True)
        self.df['Name of duration'] = self.df.apply(getDurationNamesForMongoQueries, axis=1)
        
        
        print self.df.columns
        
    def fixMultipleRestProblem(self):
        
        #Index([u'Chord name', u'Duration as a fraction of a whole note', u'Instrument name', u'Location of note or rest', u'Measure', u'Note name', u'Octave', u'Rest or note flag', u'accidental', u'duration', u'instrument', u'moveTieForFix', u'tieFlag', u'valueToAddToTiedNote', u'Name of duration'], dtype='object')
        
        
        df = pd.DataFrame(columns=self.df.columns, index=range(0, len(self.df)))
        
        cummulativeDuration = 0
        theList = []
        for index, row in self.df.iterrows():
            dictionary = {}
            if row['Rest or note flag'] == 'rest':
                cummulativeDuration = cummulativeDuration + int(row['duration'])

            if cummulativeDuration == 0 and row['Rest or note flag'] == 'note':
                dictionary["Chord name"] = row["Chord name"]
                dictionary['duration'] = row['duration']
                dictionary['Rest or note flag'] = row['Rest or note flag']
                dictionary['Instrument name'] = row['Instrument name']
                dictionary['Note name'] = row['Note name']
                theList.append(dictionary)
            
            if cummulativeDuration > 0 and row['Rest or note flag'] == 'note':
                dictionary["Chord name"] = row["Chord name"]
                dictionary['duration'] = cummulativeDuration
                cummulativeDuration = 0
                dictionary['Rest or note flag'] = 'rest'
                dictionary['Instrument name'] = row['Instrument name']
                dictionary['Note name'] = None
                theList.append(dictionary)
                
                dictionary = {}
                dictionary["Chord name"] = row["Chord name"]
                dictionary['duration'] = row['duration']
                dictionary['Rest or note flag'] = row['Rest or note flag']
                dictionary['Instrument name'] = row['Instrument name']
                dictionary['Note name'] = row['Note name']
                theList.append(dictionary)
                
            
                

        dataFrameWithMultipleRestsFixed = pd.DataFrame(theList)


                    
        display(HTML(dataFrameWithMultipleRestsFixed.to_html()))
        
    def addMetaDataAndClean(self):
        self.df['Performer name'] = self.performerName
        self.df['Composer name'] = self.composerName
        self.df['Genre'] = self.genre
        self.df['Title'] = self.title
        self.df['Year of composition'] = self.yearOfComposition
        self.df['Year of performance'] = self.yearOfPerformance
        self.df['Tempo at this location'] = self.ongoingTempo
        self.df['Title'] = self.title
        self.df['accidental'] = self.df.apply(cleanAccidental, axis=1) 
        self.df['Midi number'] = self.df.apply(midiNumberAssign, axis=1)
        self.df['Name of note'] = self.df.apply(concatenateNoteNamesAndAccidentals, axis=1)
        self.df['Name of note without octave'] = self.df.apply(concatenateNoteNamesAndAccidentalsNoOctave, axis=1)
        self.df['convertAllSharpsToFlats'] = self.df.apply(fixAccidentalEquivalencies, axis=1)
        self.df['convertAllSharpsToFlatsNoOctave'] = self.df.apply(fixAccidentalEquivalenciesNoOctave, axis=1)
        self.df['Name of note'] = self.df['convertAllSharpsToFlats']
        self.df['Name of note without octave'] = self.df['convertAllSharpsToFlatsNoOctave']
        self.df['Text written on score'] = None
        self.df['Link to audio file'] = None
        self.df['Time position in audio file where note appears'] = None
        self.df['Global location'] = None
        self.df['Structural note flag'] = None
        self.df['Auxillary note flag'] = None
        self.df['User defined search tags'] = None
        
        del self.df["Note name"]
        del self.df["accidental"]
        del self.df["duration"]
        del self.df["instrument"]
        del self.df["moveTieForFix"]
        del self.df["tieFlag"]
        del self.df["convertAllSharpsToFlatsNoOctave"]
        del self.df["convertAllSharpsToFlats"]
        del self.df["valueToAddToTiedNote"]
        
    def sortByInstrumentsAndCalculateMidiDistancesBetweenNotes(self):
        
        soloPart = self.df[self.df['Instrument name'] == 'Piano Solo Jazz (Treble)']
        soloPart = soloPart.reset_index(drop=True)
        harmonyPart = self.df[self.df['Instrument name'] == 'Jazz Harmony Bass Note']
        harmonyPart = harmonyPart.reset_index(drop=True)
        
        soloPart['Semitone distance between current and next note'] = soloPart['Midi number'].shift(-1)
        soloPart['Semitone distance between current and previous note'] = soloPart['Midi number'].shift(1)
        soloPart['Semitone distance between current and next note'] = soloPart['Semitone distance between current and next note'] - soloPart['Midi number']
        soloPart['Semitone distance between current and previous note'] = soloPart['Semitone distance between current and previous note'] - soloPart['Midi number']
        
        harmonyPart['Semitone distance between current and next note'] = harmonyPart['Midi number'].shift(-1)
        harmonyPart['Semitone distance between current and previous note'] = harmonyPart['Midi number'].shift(1)
        harmonyPart['Semitone distance between current and next note'] = harmonyPart['Semitone distance between current and next note'] - harmonyPart['Midi number']
        harmonyPart['Semitone distance between current and previous note'] = harmonyPart['Semitone distance between current and previous note'] - harmonyPart['Midi number']
        
        self.df = pd.concat([soloPart, harmonyPart], axis=0)
        self.df = self.df.sort(['Location of note or rest', 'Instrument name'], ascending=[1,1])
        self.df = self.df.reset_index(drop=True)
        
        
    
    def updateChordData(self):
        self.df['Chord name'].fillna(method='ffill', inplace=True)
        self.df['Chord root as minimized midi number'] = self.df.apply(getChordRootAsInt, axis=1)
        self.df['Chord bass as minimized midi number'] = self.df.apply(chordBassAsInt, axis=1)
        self.df['Chord type name'] = self.df.apply(chordTypeForDisplay, axis=1)
        

    def createStringsNeededForMongoRegexQueries(self):
        
        soloPart = self.df[self.df['Instrument name'] == 'Piano Solo Jazz (Treble)']
        soloPart = soloPart.reset_index(drop=True)
        harmonyPart = self.df[self.df['Instrument name'] == 'Jazz Harmony Bass Note']
        harmonyPart = harmonyPart.reset_index(drop=True)
     
        allMidiNumbersInSoloPart = soloPart['Midi number'].values
        allMidiNumbersInHarmonyPart = harmonyPart['Midi number'].values
        
        allDurationsInSoloPart = soloPart['Duration as a fraction of a whole note'].values
        allDurationsInHarmonyPart = harmonyPart['Duration as a fraction of a whole note'].values
        
        allLocationsOfRestsAndNotesInSoloPart = soloPart['Location of note or rest'].values
        allLocationsOfRestsAndNotesInHarmonyPart = harmonyPart['Location of note or rest'].values
        

        soloPart['Next fifty midi numbers'] = soloPart.apply(partial(getNextFiftyMidiNumbers,allMidiNumbersInSoloPart), axis=1)
        harmonyPart['Next fifty midi numbers'] = harmonyPart.apply(partial(getNextFiftyMidiNumbers,allMidiNumbersInHarmonyPart), axis=1)
        
        soloPart['Next fifty durations'] = soloPart.apply(partial(getNextFiftyDurations, allDurationsInSoloPart), axis=1)
        harmonyPart['Next fifty durations'] = harmonyPart.apply(partial(getNextFiftyDurations, allDurationsInHarmonyPart), axis=1)
                
        soloPart['Next fifty music event locations'] = soloPart.apply(partial(getNextFiftyLocations, allLocationsOfRestsAndNotesInSoloPart), axis=1)
        harmonyPart['Next fifty music event locations'] = harmonyPart.apply(partial(getNextFiftyLocations, allLocationsOfRestsAndNotesInHarmonyPart), axis=1) 
        
        
        soloPart['Next fifty midi distances'] = soloPart.apply(partial(getNextFiftyNotesAsDistancesFromStartingNote,allMidiNumbersInSoloPart), axis=1)
        harmonyPart['Next fifty midi distances'] = harmonyPart.apply(partial(getNextFiftyNotesAsDistancesFromStartingNote, allMidiNumbersInHarmonyPart), axis=1) 
       

        self.df = pd.concat([soloPart, harmonyPart], axis=0)
        self.df = self.df.sort(['Location of note or rest', 'Instrument name'], ascending=[1,1])
        self.df = self.df.reset_index(drop=True)
        
        

    def returnPreparedDataFrame(self):
        return self.df

In [106]:
stellaData = CreateDataSetFromXMLV2(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettStellaByStarlight.xml", 
                                  performerName="Keith Jarrett", composerName="Victor Young", genre="Jazz", 
                                  numberOfParts=2, title="Stella By Starlight", 
                                  yearOfComposition=1944, yearOfPerformance=1983, 
                                  ongoingTempo=151,numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 20000)
stellaData.convertXMLToDataFrame()
stellaData.fixLocationAndDuration()
stellaData.fixMultipleRestProblem()
# stellaData.addMetaDataAndClean()
# stellaData.sortByInstrumentsAndCalculateMidiDistancesBetweenNotes()
# stellaData.updateChordData()
# stellaData.createStringsNeededForMongoRegexQueries()
stellaDf = stellaData.returnPreparedDataFrame()

Index([u'Chord name', u'Duration as a fraction of a whole note', u'Instrument name', u'Location of note or rest', u'Measure', u'Note name', u'Octave', u'Rest or note flag', u'accidental', u'duration', u'instrument', u'moveTieForFix', u'tieFlag', u'valueToAddToTiedNote', u'Name of duration'], dtype='object')


,Chord name,Instrument name,Note name,Rest or note flag,duration
0,E min7b5,Jazz Harmony Bass Note,E,note,1024
1,None,Piano Solo Jazz (Treble),G,note,128
2,None,Piano Solo Jazz (Treble),None,rest,256
3,None,Piano Solo Jazz (Treble),A,note,128
4,None,Piano Solo Jazz (Treble),B,note,256
5,None,Piano Solo Jazz (Treble),A,note,256
6,A dom7,Jazz Harmony Bass Note,A,note,1024
7,None,Piano Solo Jazz (Treble),None,rest,128
8,None,Piano Solo Jazz (Treble),G,note,128
9,None,Piano Solo Jazz (Treble),B,note,256


In [104]:
stellaDf

,Chord name,Duration as a fraction of a whole note,Instrument name,Location of note or rest,Measure,Note name,Octave,Rest or note flag,accidental,duration,instrument,moveTieForFix,tieFlag,valueToAddToTiedNote,Name of duration
0,E min7b5,1,Jazz Harmony Bass Note,0,1,E,2,note,None,1024,P2-I1,NaN,None,NaN,Whole note
1,None,1/8,Piano Solo Jazz (Treble),0,1,G,4,note,None,128,P1-I1,0,None,0,Eighth note
2,None,1/8,Piano Solo Jazz (Treble),1/8,1,None,NaN,rest,None,128,P1-I1,0,None,0,Eighth note
3,None,1/8,Piano Solo Jazz (Treble),1/4,1,None,NaN,rest,None,128,P1-I1,0,None,0,Eighth note
4,None,1/8,Piano Solo Jazz (Treble),3/8,1,A,4,note,None,128,P1-I1,0,None,0,Eighth note
5,None,1/4,Piano Solo Jazz (Treble),5/8,1,B,4,note,<alter>-1</alter>,256,P1-I1,0,None,0,Quarter note
6,None,1/4,Piano Solo Jazz (Treble),7/8,1,A,4,note,None,256,P1-I1,0,None,0,Quarter note
7,A dom7,1,Jazz Harmony Bass Note,1,2,A,2,note,None,1024,P2-I1,NaN,None,NaN,Whole note
8,None,1/8,Piano Solo Jazz (Treble),1,2,None,NaN,rest,None,128,P1-I1,0,None,0,Eighth note
9,None,1/8,Piano Solo Jazz (Treble),9/8,2,G,4,note,None,128,P1-I1,0,None,0,Eighth note


In [130]:
armandosRhumbaData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/ChickCoreaArmandosRhumba.xml", 
                                  performerName="Chick Corea", composerName="Chick Corea", genre="Jazz", 
                                  numberOfParts=2, title="Armandos Rhumba", 
                                  yearOfComposition=1976, yearOfPerformance=1976, 
                                  ongoingTempo=261, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=4, globalLocation = 10000)
armandosRhumbaData.convertXMLToDataFrame()
armandosRhumbaData.cleanDataFrame()
armandosRhumbaData.fixLocation()
armandosRhumbaData.updateChordNumericalData()
armandosRhumbaData.removeUnusedColumns()
armandosRhumbaData.fillChordVals()
armandosRhumbaData.createStringsNeededForMongo()
armandosRhumbaDf = armandosRhumbaData.returnPreparedDataFrame()

In [51]:
groovinData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettGroovinHigh.xml", 
                                  performerName="Keith Jarrett", composerName="Dizzy Gillespie", genre="Jazz", 
                                  numberOfParts=2, title="Groovin High", 
                                  yearOfComposition=1945, yearOfPerformance=1999, 
                                  ongoingTempo=282, 
                                  numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 30000)
groovinData.convertXMLToDataFrame()
groovinData.cleanDataFrame()
groovinData.fixLocation()
groovinData.updateChordNumericalData()
groovinData.removeUnusedColumns()
groovinData.fillChordVals()
groovinData.createStringsNeededForMongo()
groovinDf = groovinData.returnPreparedDataFrame()

In [12]:
autumnData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAutumnLeaves.xml", 
                                  performerName="Keith Jarrett", composerName="Joseph Kosma", genre="Jazz", 
                                  numberOfParts=2, title="Autumn Leaves", 
                                  yearOfComposition=1945, yearOfPerformance=1996, 
                                  ongoingTempo=214, 
                                  numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 40000)
autumnData.convertXMLToDataFrame()
autumnData.cleanDataFrame()
autumnData.fixLocation()
autumnData.updateChordNumericalData()
autumnData.removeUnusedColumns()
autumnData.fillChordVals()
autumnData.createStringsNeededForMongo()
autumnDf = autumnData.returnPreparedDataFrame()

In [13]:
ifIWereData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettIfIWereABell.xml", 
                                  performerName="Keith Jarrett", composerName="Frank Loesser", genre="Jazz", 
                                  numberOfParts=2, title="If I Were A Bell", 
                                  yearOfComposition=1950, yearOfPerformance=2002, 
                                  ongoingTempo=154, numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 50000)
ifIWereData.convertXMLToDataFrame()
ifIWereData.cleanDataFrame()
ifIWereData.fixLocation()
ifIWereData.updateChordNumericalData()
ifIWereData.removeUnusedColumns()
ifIWereData.fillChordVals()
ifIWereData.createStringsNeededForMongo()
ifIWereDf = ifIWereData.returnPreparedDataFrame()

In [14]:
allTheThingsData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAllTheThingsYouAre.xml", 
                                  performerName="Keith Jarrett", composerName="Jerome Kern", genre="Jazz", 
                                  numberOfParts=2, title="All The Things You Are", 
                                  yearOfComposition=1939, yearOfPerformance=1983, 
                                  ongoingTempo=120, tempoChangeIsOccuring = False, 
                                  numberOfQuarterBeatsInBar=4, globalLocation = 60000)
allTheThingsData.convertXMLToDataFrame()
allTheThingsData.cleanDataFrame()
allTheThingsData.fixLocation()
allTheThingsData.updateChordNumericalData()
allTheThingsData.removeUnusedColumns()
allTheThingsData.fillChordVals()
allTheThingsData.createStringsNeededForMongo()
allTheThingsDf = allTheThingsData.returnPreparedDataFrame()

In [15]:
somedayMyPrinceData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettSomedayMyPrinceWillCome.xml", 
                                  performerName="Keith Jarrett", composerName="Frank Churchill", genre="Jazz", 
                                  numberOfParts=2, title="Someday My Prince Will Come", 
                                  yearOfComposition=1937, yearOfPerformance=2002, 
                                  ongoingTempo=146, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=3, globalLocation = 70000)
somedayMyPrinceData.convertXMLToDataFrame()
somedayMyPrinceData.cleanDataFrame()
somedayMyPrinceData.fixLocation()
somedayMyPrinceData.updateChordNumericalData()
somedayMyPrinceData.removeUnusedColumns()
somedayMyPrinceData.fillChordVals()
somedayMyPrinceData.createStringsNeededForMongo()
somedayMyPrinceDf = somedayMyPrinceData.returnPreparedDataFrame()

In [16]:
daysOfWineData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettDaysOfWineAndRoses.xml", 
                                  performerName="Keith Jarrett", composerName="Henry Mancini", genre="Jazz", 
                                  numberOfParts=2, title="Days Of Wine And Roses", 
                                  yearOfComposition=1962, yearOfPerformance=1994, 
                                  ongoingTempo=168, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=4, globalLocation = 80000)
daysOfWineData.convertXMLToDataFrame()
daysOfWineData.cleanDataFrame()
daysOfWineData.fixLocation()
daysOfWineData.updateChordNumericalData()
daysOfWineData.removeUnusedColumns()
daysOfWineData.fillChordVals()
daysOfWineData.createStringsNeededForMongo()
daysOfWineDf = daysOfWineData.returnPreparedDataFrame()

In [17]:
df = pd.concat([stellaDf, groovinDf, autumnDf, ifIWereDf, allTheThingsDf, somedayMyPrinceDf, daysOfWineDf, armandosRhumbaDf], axis=0)
#df = stellaDf
df.to_pickle('data_set_as_pk.pkl')

In [18]:
df['Title'].value_counts()

All The Things You Are         2313
If I Were A Bell               2280
Groovin High                   2118
Someday My Prince Will Come    2049
Stella By Starlight            1663
Days Of Wine And Roses         1614
Autumn Leaves                  1416
Armandos Rhumba                 515
dtype: int64

In [6]:
#df.head(100)

In [20]:
#df['Duration as a fraction of a whole note'].value_counts()

In [7]:
#df.dtypes

In [22]:
df.to_json(orient='records', path_or_buf='FinalDataSet/JSONFilesLoadedIntoDataBase/fullcorpus.json')